# Facial Landmark Detection

Notebook by [Prashant Brahmbhatt](https://www.github.com/hashbanger)

___

The facial landmarks are used to detect the features and different regions of a face namely:  
- Eyes 
- Eyebrows
- Nose
- Mouth
- Jawline  

Detecting facial landmarks is a 'subset' of the **shape prediction problem**. A shape predictor localizes the key points of interests along with the shape. The motive is to detect import facial features using the shape prediction methods.  
It involves two process:  
- Loacalizing the face in the image  
- Detecting the features in ROI (Region Of Interest)  


 


#### Localizing the Face

We can use the traditional Haar-Cascades to localize the face in the image. We can use a pretrained model for such purposes. The method isn't the aim but somehow we have to get a bounding box for the face.

#### Detecting facial features

There are several facial features detectors but most of them try to localize the following features:  
- Left eye
- Right eye
- Left eyebrow
- Right eyebrow
- Nose
- Jaw  
  
The *dlib* library has a facial features detector included is based on the research paper found [here](http://www.nada.kth.se/~sullivan/Papers/Kazemi_cvpr14.pdf).  

The method involves:  
- A training set of labeled facial landmarks on an image. These images are manually labeled, specifying specific (x, y)-coordinates of regions surrounding each facial structure.
- Priors, of more specifically, the probability on distance between pairs of input pixels.  

Given this training data, an ensemble of regression trees are trained to estimate the facial landmark positions directly from the pixel intensities themselves, there's no requirement of feature extraction.  
The end result is a facial landmark detector that can be used to detect facial landmarks in real-time with high quality predictions.   

### The dlib's facial detection

There is a pretrained facial landmarks detector inside the dlib library which estimates the location of 68 coordinates that map to the facial structure.  

![img1](img01.jpg)

These annotations are part of the 68 point iBUG 300-W dataset which the dlib facial landmark predictor was trained on.
Other than this there are several other models that exist as the one trained on the known HELEN dataset.    

Dlib framework can used to train for own custom shape detetcion purposes as well.

## Detecting facial landmarks using Dlib ad OpenCV

Now we create file *facial_landmarks.py* and write the code

In [ ]:
from imutils import face_utils
import numpy as np
import argparse
import imutils
import dlib
import cv2

# contructing the argument parser for parsing the arguments
ap = argparse.ArgumentParser()
ap.add_argument("-p", '--shape-predictor', required = True,
               help = 'path to facial landmark predictor')
ap.add_argument("-i", '--image', required = True,
               help = 'path to input image')
args = vars(ap.parse_args())

Here we are using the imutil's face_util to access our helper functions above.  
We will then import dlib.   

Parsing the arguments.   
- --shape-predictor : This is the path to dlib’s pre-trained facial landmark detector. You can download the detector model here

- --image : The path to the input image that we want to detect facial landmarks on

Now we initialize the dlib detector and facial landmark predictor.

In [ ]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(args["shape_predictor"])

then we load the facial landmark predictor using the path to the supplied *--shape-predictor*

First we detect the facial landmark in our image.

In [ ]:
image = cv2.imread(args['image'])
image = imutils.resize(image ,width=500)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

#detecting the faces in grayscale image
rects = detector(gray, 1)

First we load our input image from disk, then pre-processes the image by resizing to have a width of 500 pixels and converting it to grayscale.  

Then detecting the bounding box of faces in our image.  
The first parameter for our detector is our gray image and the second is the number of [image pyramid](https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_pyramids/py_pyramids.html) layers to apply when upscaling before we apply the detector (same as computing cv2.pyrUp N number of times). 

Since we have the (x, y) coordinates of the faces in the image, we can now apply facial landmark detection to each of the face regions.

In [ ]:
for (i, rect) in enumerate(rects):
    shape = predictor(gray, rect)
    shape = face_utils.shape_to_np(shape)
    
    # converting the dlib's rectangle to our traditional bounding box
    (x, y, w, h) = face_utils.rect_to_bb(rect)
    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
    
    #show the face number
    cv2.putText(image, "Face #{}".format(i +1), (x - 10, y - 10),
               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
    for (x, y) in shape:
        cv2.circle(image, (x,y), 1, (0, 0, 255), -1)
        
    cv2.imshow("Output", image)
    cv2.waitKey(0)    

Now to run the program we run the script as:  

    *python facial_landmarks.py --shape-predictor shape_predictor_68_face_landmarks.dat --image images/image1.jpg*

**References:**  
[www.medium.com ]()  
[www.pyimagesearch.com]()    
[www.stackoverflow.com]()